In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv
/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip


In [3]:
!unzip -o /kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip
!unzip -o /kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip
!unzip -o /kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip

Archive:  /kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip
  inflating: train.csv               
Archive:  /kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip
  inflating: features.csv            
Archive:  /kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip
  inflating: test.csv                


In [4]:
train = pd.read_csv('/kaggle/working/test.csv')
stores = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv')
features = pd.read_csv('/kaggle/working/features.csv')

df = pd.merge(train, features, on=['Store', 'Date'], how='left')
df = pd.merge(df, stores, on='Store', how='left')
df = df.drop('IsHoliday_y', axis=1)
df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Store,Dept,Date,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,A,151315
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,A,151315
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,A,151315
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,A,151315
4,1,1,2012-11-30,False,52.34,3.207,2460.03,NaN,3838.35,150.57,6966.34,223.610984,6.573,A,151315


In [5]:
df.shape

(115064, 15)

In [6]:
!pip install lightgbm dagshub -Uq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires dacite>=1.8, but you have dacite 1.6.0 which is incompatible.


In [7]:
!pip install mlflow==2.22.0 -Uq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 58.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 75.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
d

In [8]:
import dagshub
dagshub.init(repo_owner='Cimbir', repo_name='Store-Sales-Forecasting', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=8c1fa2e3-2883-4e9c-b21e-e34f696202be&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=a8ee1447e56af34562796d654eafb20a32934e2dbada3edaaae08c2dbc925db9




Output()

Accessing as vvaza22

Initialized MLflow to track repo "Cimbir/Store-Sales-Forecasting"

Repository Cimbir/Store-Sales-Forecasting initialized!

In [9]:
import mlflow
logged_model = 'runs:/56be70a46c5e4855a646b5759b07eb59/LightGBM Model'

loaded_model = mlflow.lightgbm.load_model(logged_model)

In [10]:
import mlflow
logged_model = 'runs:/c7bfab9f9b4e4f79bc7f3146e3768993/LightGBM_Preprocessor'

# Load model
pipeline = mlflow.sklearn.load_model(logged_model)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 1.6.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
df["Date"] = pd.to_datetime(df["Date"])

In [12]:
X_test = pipeline.transform(df)

In [13]:
X_test.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Store,Dept,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,...,Weekly_Sales_Lag_357,Weekly_Sales_Lag_364,Weekly_Sales_Lag_371,Year,Month,Day,Group_Mean_Weekly_Sales,Group_Median_Weekly_Sales,Group_Max_Weekly_Sales,Group_Min_Weekly_Sales
0,1,1,0,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,...,18689.54,39886.06,31579.90,2012,11,2,22513.322937,18535.48,57592.12,14537.37
1,1,1,0,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,...,19050.66,18689.54,39886.06,2012,11,9,22513.322937,18535.48,57592.12,14537.37
2,1,1,0,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,...,20911.25,19050.66,18689.54,2012,11,16,22513.322937,18535.48,57592.12,14537.37
3,1,1,1,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,...,25293.49,20911.25,19050.66,2012,11,23,22513.322937,18535.48,57592.12,14537.37
4,1,1,0,52.34,3.207,2460.03,NaN,3838.35,150.57,6966.34,...,33305.92,25293.49,20911.25,2012,11,30,22513.322937,18535.48,57592.12,14537.37


In [14]:
import lightgbm as lgb

test_dataset = lgb.Dataset(X_test)

In [15]:
y_pred = loaded_model.predict(X_test)

In [16]:
y_pred.shape

(115064,)

In [17]:
df.shape

(115064, 15)

In [18]:
M = df.copy()
M['Weekly_Sales'] = y_pred

In [19]:
M['Id'] = df['Store'].astype(str) + "_" + df['Dept'].astype(str) + "_" + df['Date'].astype(str)

In [20]:
M.head(10)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Store,Dept,Date,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Weekly_Sales,Id
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,A,151315,32436.463624,1_1_2012-11-02
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,A,151315,21377.123362,1_1_2012-11-09
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,A,151315,19862.452618,1_1_2012-11-16
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,A,151315,20944.776102,1_1_2012-11-23
4,1,1,2012-11-30,False,52.34,3.207,2460.03,NaN,3838.35,150.57,6966.34,223.610984,6.573,A,151315,21664.254243,1_1_2012-11-30
5,1,1,2012-12-07,False,64.12,3.198,6343.16,NaN,270.00,2928.90,10147.90,223.660021,6.573,A,151315,26793.697323,1_1_2012-12-07
6,1,1,2012-12-14,False,48.89,3.168,3504.83,NaN,73.26,1636.80,2779.60,223.719277,6.573,A,151315,37204.296716,1_1_2012-12-14
7,1,1,2012-12-21,False,56.02,3.098,8231.71,NaN,274.00,358.15,2834.02,223.839845,6.573,A,151315,39140.052893,1_1_2012-12-21
8,1,1,2012-12-28,True,44.79,3.108,12659.55,37101.13,174.78,74.46,1208.86,223.960414,6.573,A,151315,23317.975408,1_1_2012-12-28
9,1,1,2013-01-04,False,41.73,3.161,1214.08,25366.33,15.01,72.36,3940.02,224.080983,6.525,A,151315,18340.902647,1_1_2013-01-04


In [21]:
submission = M[['Id', 'Weekly_Sales']].copy()

In [22]:
submission.head()

,Id,Weekly_Sales
0,1_1_2012-11-02,32436.463624
1,1_1_2012-11-09,21377.123362
2,1_1_2012-11-16,19862.452618
3,1_1_2012-11-23,20944.776102
4,1_1_2012-11-30,21664.254243


In [23]:
submission.tail()

,Id,Weekly_Sales
115059,45_98_2013-06-28,693.281846
115060,45_98_2013-07-05,678.145626
115061,45_98_2013-07-12,649.778820
115062,45_98_2013-07-19,716.793140
115063,45_98_2013-07-26,717.762273


In [24]:
submission.to_csv("submission.csv", index=False)